In [ ]:
#importing libraries
import cv2
import numpy as np

#intialize the camera
camera = cv2.VideoCapture(0)

# create a window for the trackbar
cv2.namedWindow("Threshold adjust")
cv2.resizeWindow("Threshold adjust",640,240)
def empty(n):
    pass
cv2.createTrackbar("Threshold","Threshold adjust",40,255,empty)
#get the frames for the difference
ret,frame1 = camera.read()
ret,frame2 = camera.read()

# start the capturing loop
while True:
    # store the copy for the drawing of detection
    frame = frame1.copy()
    
    # get the difference from the images
    diff = cv2.absdiff(frame1,frame2)
    
    # get the gray scale
    gray = cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)
    
    # blur the image for the noise removal
    blur = cv2.GaussianBlur(gray,(5,5),0)
    
    # get the trackbar current value
    value = cv2.getTrackbarPos("Threshold","Threshold adjust")
    
    # apply thresholding
    _,thresh = cv2.threshold(blur,value,255,cv2.THRESH_BINARY)
    
    #find the continues points
    _,contours,_ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #draw the rectangle around the detection of motion
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        # if any area is less than 900 pixels ignore it
        if cv2.contourArea(contour) <900:
            continue
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
        
    #to draw the original contours uncomment it 
    #cv2.drawContours(frame,contours,-1,(0,255,0),2)
    # display all the difference,frame and threshold
    cv2.imshow("difference",diff)
    cv2.imshow("frame",frame)
    cv2.imshow("threshold",thresh)
    
    frame2 = frame1
    ret,frame1 = camera.read()
    
    # break the loop if any interept occur with escape key
    if cv2.waitKey(100) & 0xFF == 27:
        break

# after the loop destroy the camera release and the windows will be destroyed
camera.release()
cv2.destroyAllWindows()